In [2]:
import os
import numpy as np
import cv2
from PIL import Image
import cairosvg
import io
from collections import Counter
import colorsys
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

class FlagFeatureExtractor:
    def __init__(self, target_size=(128, 64)):
        self.target_size = target_size
        
    def load_flag_from_svg(self, svg_path, remove_padding=True):
        """Load and preprocess flag from SVG"""
        png_bytes = cairosvg.svg2png(
            url=svg_path, 
            output_width=self.target_size[0], 
            output_height=self.target_size[1]
        )
        img = Image.open(io.BytesIO(png_bytes)).convert("RGB")
        img_array = np.array(img) / 255.0
        
        if remove_padding:
            img_array = self.remove_black_padding(img_array)
            
        return img_array
    
    def remove_black_padding(self, img_array, threshold=0.05):
        """Remove black padding from flag image"""
        # Find non-black pixels
        mask = np.any(img_array > threshold, axis=2)
        
        if not np.any(mask):
            return img_array
            
        # Find bounding box
        coords = np.argwhere(mask)
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0)
        
        # Crop to content
        return img_array[y0:y1+1, x0:x1+1]
    
    def extract_color_features(self, img_array):
        """Extract color-based features"""
        features = {}
        
        # Reshape for analysis
        pixels = img_array.reshape(-1, 3)
        
        # Dominant colors using K-means
        n_colors = min(8, len(np.unique(pixels.reshape(-1, pixels.shape[-1]), axis=0)))
        if n_colors > 1:
            kmeans = KMeans(n_clusters=n_colors, random_state=42, n_init=10)
            kmeans.fit(pixels)
            dominant_colors = kmeans.cluster_centers_
            color_proportions = np.bincount(kmeans.labels_) / len(kmeans.labels_)
        else:
            dominant_colors = [pixels[0]]
            color_proportions = [1.0]
        
        # Color categories
        color_categories = []
        for color in dominant_colors:
            h, s, v = colorsys.rgb_to_hsv(*color)
            category = self.categorize_color(h, s, v)
            color_categories.append(category)
        
        features['dominant_colors'] = dominant_colors
        features['color_proportions'] = color_proportions
        features['color_categories'] = Counter(color_categories)
        features['num_colors'] = len(dominant_colors)
        
        # Color distribution statistics
        features['brightness'] = np.mean(pixels)
        features['color_variance'] = np.var(pixels)
        
        return features
    
    def categorize_color(self, h, s, v):
        """Categorize RGB color into semantic categories"""
        if s < 0.2:  # Low saturation
            if v < 0.3:
                return 'black'
            elif v > 0.8:
                return 'white'
            else:
                return 'gray'
        
        # High saturation colors by hue
        if h < 0.05 or h > 0.95:
            return 'red'
        elif 0.05 <= h < 0.15:
            return 'orange'
        elif 0.15 <= h < 0.25:
            return 'yellow'
        elif 0.25 <= h < 0.4:
            return 'green'
        elif 0.4 <= h < 0.65:
            return 'blue'
        elif 0.65 <= h < 0.8:
            return 'purple'
        else:
            return 'pink'
    
    def extract_geometric_features(self, img_array):
        """Extract geometric and structural features"""
        features = {}
        
        # Convert to grayscale for edge detection
        if len(img_array.shape) == 3:
            gray = cv2.cvtColor((img_array * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            gray = (img_array * 255).astype(np.uint8)
        
        # Edge detection
        edges = cv2.Canny(gray, 50, 150)
        features['edge_density'] = np.sum(edges > 0) / edges.size
        
        # Line detection
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=20, minLineLength=10, maxLineGap=5)
        
        if lines is not None:
            features['num_lines'] = len(lines)
            
            # Classify line orientations
            horizontal_lines = 0
            vertical_lines = 0
            diagonal_lines = 0
            
            for line in lines:
                x1, y1, x2, y2 = line[0]
                angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
                
                if angle < 15 or angle > 165:
                    horizontal_lines += 1
                elif 75 < angle < 105:
                    vertical_lines += 1
                else:
                    diagonal_lines += 1
            
            features['horizontal_lines'] = horizontal_lines
            features['vertical_lines'] = vertical_lines
            features['diagonal_lines'] = diagonal_lines
        else:
            features['num_lines'] = 0
            features['horizontal_lines'] = 0
            features['vertical_lines'] = 0
            features['diagonal_lines'] = 0
        
        # Contour analysis for shapes
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        features['num_shapes'] = len(contours)
        
        # Analyze shape complexity
        if contours:
            areas = [cv2.contourArea(c) for c in contours if cv2.contourArea(c) > 10]
            perimeters = [cv2.arcLength(c, True) for c in contours if cv2.contourArea(cv2.approxPolyDP(c, 0.02*cv2.arcLength(c, True), True)) > 10]
            
            if areas and perimeters:
                features['avg_shape_complexity'] = np.mean([p**2/(4*np.pi*a) if a > 0 else 0 for a, p in zip(areas, perimeters)])
                features['shape_size_variance'] = np.var(areas) if len(areas) > 1 else 0
            else:
                features['avg_shape_complexity'] = 0
                features['shape_size_variance'] = 0
        
        return features
    
    def extract_pattern_features(self, img_array):
        """Extract pattern-based features"""
        features = {}
        
        # Stripe detection
        features.update(self.detect_stripes(img_array))
        
        # Symmetry analysis
        features.update(self.analyze_symmetry(img_array))
        
        # Quarter analysis (for crosses, unions, etc.)
        features.update(self.analyze_quarters(img_array))
        
        return features
    
    def detect_stripes(self, img_array):
        """Detect horizontal and vertical stripes"""
        features = {}
        h, w = img_array.shape[:2]
        
        # Horizontal stripe detection
        row_colors = []
        for i in range(h):
            row_avg = np.mean(img_array[i, :], axis=0)
            row_colors.append(row_avg)
        
        # Count color changes (indicates stripes)
        horizontal_changes = 0
        for i in range(1, len(row_colors)):
            if np.linalg.norm(row_colors[i] - row_colors[i-1]) > 0.3:
                horizontal_changes += 1
        
        features['horizontal_stripes'] = horizontal_changes
        
        # Vertical stripe detection
        col_colors = []
        for j in range(w):
            col_avg = np.mean(img_array[:, j], axis=0)
            col_colors.append(col_avg)
        
        vertical_changes = 0
        for j in range(1, len(col_colors)):
            if np.linalg.norm(col_colors[j] - col_colors[j-1]) > 0.3:
                vertical_changes += 1
                
        features['vertical_stripes'] = vertical_changes
        
        # Determine stripe pattern type
        if horizontal_changes > vertical_changes and horizontal_changes > 2:
            features['stripe_type'] = 'horizontal'
        elif vertical_changes > horizontal_changes and vertical_changes > 2:
            features['stripe_type'] = 'vertical'
        elif horizontal_changes > 0 and vertical_changes > 0:
            features['stripe_type'] = 'both'
        else:
            features['stripe_type'] = 'none'
            
        return features
    
    def analyze_symmetry(self, img_array):
        """Analyze horizontal and vertical symmetry"""
        features = {}
        
        h, w = img_array.shape[:2]
        
        # Horizontal symmetry (top-bottom)
        top_half = img_array[:h//2, :]
        bottom_half = np.flip(img_array[h//2:], axis=0)
        min_h = min(top_half.shape[0], bottom_half.shape[0])
        
        if min_h > 0:
            h_symmetry = 1 - np.mean(np.abs(top_half[:min_h] - bottom_half[:min_h]))
            features['horizontal_symmetry'] = max(0, h_symmetry)
        else:
            features['horizontal_symmetry'] = 0
        
        # Vertical symmetry (left-right)
        left_half = img_array[:, :w//2]
        right_half = np.flip(img_array[:, w//2:], axis=1)
        min_w = min(left_half.shape[1], right_half.shape[1])
        
        if min_w > 0:
            v_symmetry = 1 - np.mean(np.abs(left_half[:, :min_w] - right_half[:, :min_w]))
            features['vertical_symmetry'] = max(0, v_symmetry)
        else:
            features['vertical_symmetry'] = 0
            
        return features
    
    def analyze_quarters(self, img_array):
        """Analyze flag quarters for crosses, unions, etc."""
        features = {}
        
        h, w = img_array.shape[:2]
        
        # Divide into quarters
        q1 = img_array[:h//2, :w//2]  # top-left
        q2 = img_array[:h//2, w//2:]  # top-right  
        q3 = img_array[h//2:, :w//2]  # bottom-left
        q4 = img_array[h//2:, w//2:]  # bottom-right
        
        quarters = [q1, q2, q3, q4]
        
        # Calculate average color of each quarter
        quarter_colors = [np.mean(q, axis=(0,1)) for q in quarters]
        
        # Check for cross patterns (opposite quarters similar)
        cross_similarity = (
            np.linalg.norm(quarter_colors[0] - quarter_colors[3]) + 
            np.linalg.norm(quarter_colors[1] - quarter_colors[2])
        ) / 2
        
        features['cross_pattern'] = 1 - min(cross_similarity, 1.0)
        
        # Check for union jack pattern (quarters different)
        quarter_variance = np.var([np.linalg.norm(qc) for qc in quarter_colors])
        features['quarter_diversity'] = min(quarter_variance, 1.0)
        
        return features
    
    def extract_all_features(self, img_array):
        """Extract all feature categories"""
        all_features = {}
        
        all_features.update(self.extract_color_features(img_array))
        all_features.update(self.extract_geometric_features(img_array))
        all_features.update(self.extract_pattern_features(img_array))
        
        return all_features
    
    def analyze_flag_directory(self, svg_dir):
        """Analyze all flags in directory"""
        results = {}
        
        for filename in os.listdir(svg_dir):
            if filename.endswith('.svg'):
                code = filename.split('.')[0]
                svg_path = os.path.join(svg_dir, filename)
                
                try:
                    img_array = self.load_flag_from_svg(svg_path)
                    features = self.extract_all_features(img_array)
                    results[code] = features
                    print(f"Analyzed {code}")
                except Exception as e:
                    print(f"Error analyzing {code}: {e}")
        
        return results
    
    def print_feature_summary(self, features, flag_code):
        """Print a readable summary of flag features"""
        print(f"\n=== FLAG FEATURES: {flag_code.upper()} ===")
        
        # Color features
        print(f"Colors: {features['num_colors']} dominant colors")
        print(f"Color categories: {dict(features['color_categories'])}")
        print(f"Brightness: {features['brightness']:.2f}")
        
        # Geometric features  
        print(f"Lines: {features['num_lines']} total ({features['horizontal_lines']} horiz, {features['vertical_lines']} vert)")
        print(f"Shapes: {features['num_shapes']}")
        print(f"Edge density: {features['edge_density']:.3f}")
        
        # Pattern features
        print(f"Stripe type: {features['stripe_type']}")
        print(f"Stripes: {features['horizontal_stripes']} horiz, {features['vertical_stripes']} vert")
        print(f"Symmetry: {features['horizontal_symmetry']:.2f} horiz, {features['vertical_symmetry']:.2f} vert")
        print(f"Cross pattern: {features['cross_pattern']:.2f}")

# Example usage
if __name__ == "__main__":
    # Initialize extractor
    extractor = FlagFeatureExtractor()
    
    # Analyze single flag
    svg_path = "sovereign_flag_svgs"  # Replace with actual path
    # img_array = extractor.load_flag_from_svg(svg_path)
    # features = extractor.extract_all_features(img_array)
    # extractor.print_feature_summary(features, "example")
    
    # Analyze whole directory
    # results = extractor.analyze_flag_directory("sovereign_flag_svgs")
    
    print("Flag feature extractor ready!")
    print("Use extractor.analyze_flag_directory('your_svg_dir') to analyze all flags")

Flag feature extractor ready!
Use extractor.analyze_flag_directory('your_svg_dir') to analyze all flags


In [3]:
extractor.analyze_flag_directory("sovereign_flag_svgs")

Analyzed ad
Analyzed ae
Analyzed af
Analyzed ag
Analyzed al
Analyzed am
Analyzed ao
Analyzed ar
Analyzed at
Analyzed au
Analyzed az
Analyzed ba
Analyzed bb
Analyzed bd
Analyzed be
Analyzed bf
Analyzed bg
Analyzed bh
Analyzed bi
Analyzed bj
Analyzed bn
Analyzed bo
Analyzed br
Analyzed bs
Analyzed bt
Analyzed bw
Analyzed by
Analyzed bz
Analyzed ca
Analyzed cd
Analyzed cf
Analyzed cg
Analyzed ch
Analyzed ci
Analyzed cl
Analyzed cm
Analyzed cn
Analyzed co
Analyzed cr
Analyzed cu
Analyzed cv
Analyzed cy
Analyzed cz
Analyzed de
Analyzed dj
Analyzed dk
Analyzed dm
Analyzed do
Analyzed dz
Analyzed ec
Analyzed ee
Analyzed eg
Analyzed er
Analyzed es
Analyzed et
Analyzed fi
Analyzed fj
Analyzed fm
Analyzed fr
Analyzed ga
Analyzed gb
Analyzed gd
Analyzed ge
Analyzed gh
Analyzed gm
Analyzed gn
Analyzed gq
Analyzed gr
Analyzed gt
Analyzed gw
Analyzed gy
Analyzed hn
Analyzed hr
Analyzed ht
Analyzed hu
Analyzed id
Analyzed ie
Analyzed il
Analyzed in
Analyzed iq
Analyzed ir
Analyzed is
Analyzed it
Anal

{'ad': {'dominant_colors': array([[ 8.15696760e-01,  6.35273147e-02,  2.26838628e-01],
         [-1.22124533e-15,  9.41176471e-02,  6.58823529e-01],
         [ 9.94961098e-01,  8.72664841e-01,  9.59271825e-04],
         [ 7.32956259e-01,  6.14199526e-01,  4.17539323e-01],
         [ 8.90783140e-01,  4.33814723e-01,  1.12293061e-01],
         [ 8.76930147e-01,  7.32138480e-01,  1.55238971e-01],
         [ 2.54901961e-01,  2.54901961e-01,  4.98039216e-01],
         [ 4.54901961e-01,  4.47058824e-01,  3.60784314e-01]]),
  'color_proportions': array([0.31759511, 0.30434783, 0.24439538, 0.06182065, 0.02836277,
         0.02173913, 0.01086957, 0.01086957]),
  'color_categories': Counter({'orange': 4,
           'red': 1,
           'blue': 1,
           'purple': 1,
           'yellow': 1}),
  'num_colors': 8,
  'brightness': np.float64(0.41666444657573176),
  'color_variance': np.float64(0.1355512276792395),
  'edge_density': np.float64(0.07489809782608696),
  'num_lines': 4,
  'horizontal_

In [4]:
import os
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

class FlagAnalyzer:
    def __init__(self, feature_results):
        """Initialize with results from FlagFeatureExtractor"""
        self.results = feature_results
        self.df = self.create_feature_dataframe()
    
    def create_feature_dataframe(self):
        """Convert feature results to clean pandas DataFrame"""
        rows = []
        
        for country, features in self.results.items():
            row = {
                'country': country,
                # Color features - simplified
                'num_colors': features['num_colors'],
                'brightness': round(float(features['brightness']), 3),
                'primary_colors': self.get_primary_colors(features['color_categories']),
                
                # Pattern features
                'stripe_type': features['stripe_type'],
                'num_horizontal_stripes': features['horizontal_stripes'],
                'num_vertical_stripes': features['vertical_stripes'],
                
                # Symmetry (rounded for readability)
                'h_symmetry': round(float(features['horizontal_symmetry']), 2),
                'v_symmetry': round(float(features['vertical_symmetry']), 2),
                
                # Shape features
                'num_lines': features['num_lines'],
                'num_shapes': features['num_shapes'],
                'edge_density': round(float(features['edge_density']), 3),
                
                # Special patterns
                'has_cross': float(features['cross_pattern']) > 0.8,
                'cross_strength': round(float(features['cross_pattern']), 2),
            }
            rows.append(row)
        
        return pd.DataFrame(rows)
    
    def get_primary_colors(self, color_counter):
        """Get the top 3 most common colors as a string"""
        top_colors = color_counter.most_common(3)
        return ', '.join([color for color, count in top_colors])
    
    def find_similar_flags(self, country_code, top_n=5):
        """Find flags most similar to given country"""
        if country_code not in self.df['country'].values:
            print(f"Country {country_code} not found!")
            return
        
        # Get target flag features
        target = self.df[self.df['country'] == country_code].iloc[0]
        
        # Calculate similarity scores
        similarities = []
        for _, row in self.df.iterrows():
            if row['country'] == country_code:
                continue
                
            score = self.calculate_similarity(target, row)
            similarities.append((row['country'], score))
        
        # Sort and display
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        print(f"\n🏁 FLAGS MOST SIMILAR TO {country_code.upper()}:")
        print("="*50)
        for i, (country, score) in enumerate(similarities[:top_n], 1):
            flag_info = self.df[self.df['country'] == country].iloc[0]
            print(f"{i}. {country.upper()} (similarity: {score:.2f})")
            print(f"   Colors: {flag_info['primary_colors']}")
            print(f"   Pattern: {flag_info['stripe_type']} stripes")
            print(f"   Symmetry: H={flag_info['h_symmetry']}, V={flag_info['v_symmetry']}")
            print()
    
    def calculate_similarity(self, flag1, flag2):
        """Calculate similarity between two flags"""
        score = 0
        
        # Color similarity
        if flag1['primary_colors'] == flag2['primary_colors']:
            score += 3
        elif any(color in flag2['primary_colors'] for color in flag1['primary_colors'].split(', ')):
            score += 1
        
        # Pattern similarity
        if flag1['stripe_type'] == flag2['stripe_type'] and flag1['stripe_type'] != 'none':
            score += 2
        
        # Symmetry similarity
        score += 1 - abs(flag1['h_symmetry'] - flag2['h_symmetry'])
        score += 1 - abs(flag1['v_symmetry'] - flag2['v_symmetry'])
        
        # Cross pattern
        if flag1['has_cross'] and flag2['has_cross']:
            score += 1
        
        # Number of colors
        color_diff = abs(flag1['num_colors'] - flag2['num_colors'])
        score += max(0, 1 - color_diff/5)
        
        return score
    
    def analyze_patterns(self):
        """Analyze common patterns across all flags"""
        print("🎨 FLAG PATTERN ANALYSIS")
        print("="*50)
        
        # Stripe patterns
        stripe_counts = Counter(self.df['stripe_type'])
        print(f"\nStripe Patterns:")
        for pattern, count in stripe_counts.items():
            print(f"  {pattern.title()}: {count} flags")
        
        # Color analysis
        all_colors = []
        for colors_str in self.df['primary_colors']:
            all_colors.extend(colors_str.split(', '))
        color_popularity = Counter(all_colors)
        
        print(f"\nMost Popular Colors:")
        for color, count in color_popularity.most_common(8):
            print(f"  {color.title()}: {count} flags")
        
        # Symmetry analysis
        high_symmetry = self.df[(self.df['h_symmetry'] > 0.9) & (self.df['v_symmetry'] > 0.9)]
        print(f"\nHighly Symmetric Flags ({len(high_symmetry)}):")
        for _, flag in high_symmetry.head(10).iterrows():
            print(f"  {flag['country'].upper()}")
        
        # Cross patterns
        cross_flags = self.df[self.df['has_cross']]
        print(f"\nFlags with Cross Patterns ({len(cross_flags)}):")
        for _, flag in cross_flags.head(10).iterrows():
            print(f"  {flag['country'].upper()} (strength: {flag['cross_strength']})")
    
    def find_flag_families(self):
        """Group flags into families based on shared characteristics"""
        families = defaultdict(list)
        
        # Nordic cross family
        nordic = self.df[(self.df['has_cross']) & (self.df['cross_strength'] > 0.85)]
        if len(nordic) > 0:
            families['Nordic Cross'] = nordic['country'].tolist()
        
        # Tricolor families
        tricolors_h = self.df[(self.df['stripe_type'] == 'horizontal') & 
                             (self.df['num_horizontal_stripes'] == 2)]  # 3 stripes = 2 changes
        if len(tricolors_h) > 0:
            families['Horizontal Tricolor'] = tricolors_h['country'].tolist()
            
        tricolors_v = self.df[(self.df['stripe_type'] == 'vertical') & 
                             (self.df['num_vertical_stripes'] == 2)]
        if len(tricolors_v) > 0:
            families['Vertical Tricolor'] = tricolors_v['country'].tolist()
        
        # Red-white combinations
        red_white = self.df[self.df['primary_colors'].str.contains('red') & 
                           self.df['primary_colors'].str.contains('white')]
        if len(red_white) > 2:
            families['Red & White'] = red_white['country'].tolist()
        
        # Pan-African colors (red, green, yellow/gold)
        pan_african = self.df[self.df['primary_colors'].str.contains('red') & 
                             self.df['primary_colors'].str.contains('green') &
                             (self.df['primary_colors'].str.contains('yellow') | 
                              self.df['primary_colors'].str.contains('orange'))]
        if len(pan_african) > 0:
            families['Pan-African Colors'] = pan_african['country'].tolist()
        
        # Blue-white combinations  
        blue_white = self.df[self.df['primary_colors'].str.contains('blue') & 
                            self.df['primary_colors'].str.contains('white')]
        if len(blue_white) > 2:
            families['Blue & White'] = blue_white['country'].tolist()
        
        print("\n🏁 FLAG FAMILIES")
        print("="*50)
        for family_name, countries in families.items():
            print(f"\n{family_name} ({len(countries)} flags):")
            for country in countries:
                print(f"  {country.upper()}")
    
    def create_summary_table(self, countries=None):
        """Create a clean summary table"""
        df_display = self.df.copy()
        
        if countries:
            df_display = df_display[df_display['country'].isin(countries)]
        
        # Select and rename columns for display
        display_cols = {
            'country': 'Country',
            'primary_colors': 'Colors',
            'stripe_type': 'Stripes',
            'h_symmetry': 'H.Sym',
            'v_symmetry': 'V.Sym',
            'has_cross': 'Cross',
            'num_colors': '#Colors',
            'brightness': 'Bright'
        }
        
        summary = df_display[list(display_cols.keys())].rename(columns=display_cols)
        summary['Country'] = summary['Country'].str.upper()
        
        return summary
    
    def plot_feature_distributions(self):
        """Create visualizations of feature distributions"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Flag Feature Distributions', fontsize=16)
        
        # Color count distribution
        axes[0,0].hist(self.df['num_colors'], bins=range(1, 11), alpha=0.7)
        axes[0,0].set_title('Number of Colors')
        axes[0,0].set_xlabel('Colors')
        axes[0,0].set_ylabel('Frequency')
        
        # Brightness distribution
        axes[0,1].hist(self.df['brightness'], bins=20, alpha=0.7)
        axes[0,1].set_title('Brightness Distribution')
        axes[0,1].set_xlabel('Brightness')
        axes[0,1].set_ylabel('Frequency')
        
        # Symmetry scatter
        axes[0,2].scatter(self.df['h_symmetry'], self.df['v_symmetry'], alpha=0.6)
        axes[0,2].set_title('Symmetry: Horizontal vs Vertical')
        axes[0,2].set_xlabel('Horizontal Symmetry')
        axes[0,2].set_ylabel('Vertical Symmetry')
        
        # Stripe type counts
        stripe_counts = self.df['stripe_type'].value_counts()
        axes[1,0].bar(stripe_counts.index, stripe_counts.values)
        axes[1,0].set_title('Stripe Patterns')
        axes[1,0].tick_params(axis='x', rotation=45)
        
        # Color popularity
        all_colors = []
        for colors in self.df['primary_colors']:
            all_colors.extend(colors.split(', '))
        color_counts = Counter(all_colors)
        top_colors = dict(color_counts.most_common(8))
        
        axes[1,1].bar(top_colors.keys(), top_colors.values())
        axes[1,1].set_title('Most Popular Colors')
        axes[1,1].tick_params(axis='x', rotation=45)
        
        # Cross pattern strength
        cross_flags = self.df[self.df['has_cross']]
        if len(cross_flags) > 0:
            axes[1,2].hist(cross_flags['cross_strength'], bins=10, alpha=0.7)
            axes[1,2].set_title('Cross Pattern Strength')
            axes[1,2].set_xlabel('Cross Strength')
            axes[1,2].set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.show()

# Usage functions
def quick_analysis(results):
    """Quick analysis of flag features"""
    analyzer = FlagAnalyzer(results)
    
    print("📊 QUICK FLAG ANALYSIS")
    print("="*50)
    print(f"Total flags analyzed: {len(analyzer.df)}")
    print(f"Average colors per flag: {analyzer.df['num_colors'].mean():.1f}")
    print(f"Most symmetric flag: {analyzer.df.loc[analyzer.df['h_symmetry'].idxmax(), 'country'].upper()}")
    print(f"Brightest flag: {analyzer.df.loc[analyzer.df['brightness'].idxmax(), 'country'].upper()}")
    
    analyzer.analyze_patterns()
    analyzer.find_flag_families()
    
    return analyzer

def compare_flags(results, flag1, flag2):
    """Compare two specific flags"""
    analyzer = FlagAnalyzer(results)
    
    if flag1 not in analyzer.df['country'].values or flag2 not in analyzer.df['country'].values:
        print("One or both countries not found!")
        return
    
    f1 = analyzer.df[analyzer.df['country'] == flag1].iloc[0]
    f2 = analyzer.df[analyzer.df['country'] == flag2].iloc[0]
    
    print(f"\n🆚 COMPARING {flag1.upper()} vs {flag2.upper()}")
    print("="*50)
    
    features = ['primary_colors', 'stripe_type', 'h_symmetry', 'v_symmetry', 
                'num_colors', 'brightness', 'has_cross']
    
    for feature in features:
        print(f"{feature:15}: {f1[feature]:20} | {f2[feature]}")
    
    similarity = analyzer.calculate_similarity(f1, f2)
    print(f"\nSimilarity Score: {similarity:.2f}/10")

# Example usage
print("Flag analyzer ready!")
print("\nUsage:")
print("analyzer = quick_analysis(your_feature_results)")
print("analyzer.find_similar_flags('us')")  
print("compare_flags(your_feature_results, 'us', 'uk')")

Flag analyzer ready!

Usage:
analyzer = quick_analysis(your_feature_results)
analyzer.find_similar_flags('us')
compare_flags(your_feature_results, 'us', 'uk')


In [7]:
analyzer = quick_analysis(extractor.analyze_flag_directory("sovereign_flag_svgs"))
analyzer.find_similar_flags('in')
compare_flags(extractor.analyze_flag_directory("sovereign_flag_svgs"), 'us', 'gb')

Analyzed ad
Analyzed ae
Analyzed af
Analyzed ag
Analyzed al
Analyzed am
Analyzed ao
Analyzed ar
Analyzed at
Analyzed au
Analyzed az
Analyzed ba
Analyzed bb
Analyzed bd
Analyzed be
Analyzed bf
Analyzed bg
Analyzed bh
Analyzed bi
Analyzed bj
Analyzed bn
Analyzed bo
Analyzed br
Analyzed bs
Analyzed bt
Analyzed bw
Analyzed by
Analyzed bz
Analyzed ca
Analyzed cd
Analyzed cf
Analyzed cg
Analyzed ch
Analyzed ci
Analyzed cl
Analyzed cm
Analyzed cn
Analyzed co
Analyzed cr
Analyzed cu
Analyzed cv
Analyzed cy
Analyzed cz
Analyzed de
Analyzed dj
Analyzed dk
Analyzed dm
Analyzed do
Analyzed dz
Analyzed ec
Analyzed ee
Analyzed eg
Analyzed er
Analyzed es
Analyzed et
Analyzed fi
Analyzed fj
Analyzed fm
Analyzed fr
Analyzed ga
Analyzed gb
Analyzed gd
Analyzed ge
Analyzed gh
Analyzed gm
Analyzed gn
Analyzed gq
Analyzed gr
Analyzed gt
Analyzed gw
Analyzed gy
Analyzed hn
Analyzed hr
Analyzed ht
Analyzed hu
Analyzed id
Analyzed ie
Analyzed il
Analyzed in
Analyzed iq
Analyzed ir
Analyzed is
Analyzed it
Anal